In [1]:
!pip install findspark

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Crear la sesión de Spark
spark = SparkSession.builder.appName("PracticaSpark").getOrCreate()

In [3]:
# Cargar el archivo CSV en un DataFrame
df = spark.read.csv("./work/data/archivo_limpio.csv", header=True, inferSchema=True)

# Mostrar los primeros registros
df.show()

# Mostrar el esquema del DataFrame
df.printSchema()

# Contar el número de filas
print(f"Número de filas: {df.count()}")


+----------+----+-----+-------+--------------------+---------+
|     fecha|anio|  mes|  linea|            estacion|afluencia|
+----------+----+-----+-------+--------------------+---------+
|2010-01-01|2010|Enero|Linea 1|            Zaragoza|    20227|
|2010-01-01|2010|Enero|Linea 1| Isabel la CatA³lica|     6487|
|2010-01-01|2010|Enero|Linea 1|           Moctezuma|    10304|
|2010-01-01|2010|Enero|Linea 1|        Pino SuA¡rez|     8679|
|2010-01-01|2010|Enero|Linea 1|      GA³mez FarA­as|    19499|
|2010-01-01|2010|Enero|Linea 6|Deportivo 18 de M...|      621|
|2010-01-01|2010|Enero|Linea 6|  La Villa/BasA­lica|    24792|
|2010-01-01|2010|Enero|Linea 9|          PantitlA¡n|    27000|
|2010-01-01|2010|Enero|Linea 8|              Aculco|     3652|
|2010-01-01|2010|Enero|Linea 9|          VelA³dromo|     3239|
|2010-01-01|2010|Enero|Linea 5| Autobuses del Norte|    16824|
|2010-01-01|2010|Enero|Linea 5|           Misterios|     3513|
|2010-01-01|2010|Enero|Linea 7|      Constituyentes|   